In [1]:
# -*- coding:utf-8 -*-
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
import pickle as pkl
from datetime import *
import os

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
column_list = "label,channel_id,coin,timestamp,length,coin_seq,feature_seq,pre_1h_return,pre_1h_price,pre_1h_price_avg,pre_1h_volume,pre_1h_volume_avg,pre_1h_volume_sum,pre_1h_volume_tb,pre_1h_volume_quote,pre_1h_volume_quote_tb,pre_3h_return,pre_3h_price,pre_3h_price_avg,pre_3h_volume,pre_3h_volume_avg,pre_3h_volume_sum,pre_3h_volume_tb,pre_3h_volume_tb_avg,pre_3h_volume_tb_sum,pre_3h_volume_quote,pre_3h_volume_quote_avg,pre_3h_volume_quote_sum,pre_3h_volume_quote_tb,pre_3h_volume_quote_tb_avg,pre_3h_volume_quote_tb_sum,pre_6h_return,pre_6h_price,pre_6h_price_avg,pre_6h_volume,pre_6h_volume_avg,pre_6h_volume_sum,pre_6h_volume_tb,pre_6h_volume_tb_avg,pre_6h_volume_tb_sum,pre_6h_volume_quote,pre_6h_volume_quote_avg,pre_6h_volume_quote_sum,pre_6h_volume_quote_tb,pre_6h_volume_quote_tb_avg,pre_6h_volume_quote_tb_sum,pre_12h_return,pre_12h_price,pre_12h_price_avg,pre_12h_volume,pre_12h_volume_avg,pre_12h_volume_sum,pre_12h_volume_tb,pre_12h_volume_tb_avg,pre_12h_volume_tb_sum,pre_12h_volume_quote,pre_12h_volume_quote_avg,pre_12h_volume_quote_sum,pre_12h_volume_quote_tb,pre_12h_volume_quote_tb_avg,pre_12h_volume_quote_tb_sum,pre_24h_return,pre_24h_price,pre_24h_price_avg,pre_24h_volume,pre_24h_volume_avg,pre_24h_volume_sum,pre_24h_volume_tb,pre_24h_volume_tb_avg,pre_24h_volume_tb_sum,pre_24h_volume_quote,pre_24h_volume_quote_avg,pre_24h_volume_quote_sum,pre_24h_volume_quote_tb,pre_24h_volume_quote_tb_avg,pre_24h_volume_quote_tb_sum,pre_36h_return,pre_36h_price,pre_36h_price_avg,pre_36h_volume,pre_36h_volume_avg,pre_36h_volume_sum,pre_36h_volume_tb,pre_36h_volume_tb_avg,pre_36h_volume_tb_sum,pre_36h_volume_quote,pre_36h_volume_quote_avg,pre_36h_volume_quote_sum,pre_36h_volume_quote_tb,pre_36h_volume_quote_tb_avg,pre_36h_volume_quote_tb_sum,pre_48h_return,pre_48h_price,pre_48h_price_avg,pre_48h_volume,pre_48h_volume_avg,pre_48h_volume_sum,pre_48h_volume_tb,pre_48h_volume_tb_avg,pre_48h_volume_tb_sum,pre_48h_volume_quote,pre_48h_volume_quote_avg,pre_48h_volume_quote_sum,pre_48h_volume_quote_tb,pre_48h_volume_quote_tb_avg,pre_48h_volume_quote_tb_sum,pre_60h_return,pre_60h_price,pre_60h_price_avg,pre_60h_volume,pre_60h_volume_avg,pre_60h_volume_sum,pre_60h_volume_tb,pre_60h_volume_tb_avg,pre_60h_volume_tb_sum,pre_60h_volume_quote,pre_60h_volume_quote_avg,pre_60h_volume_quote_sum,pre_60h_volume_quote_tb,pre_60h_volume_quote_tb_avg,pre_60h_volume_quote_tb_sum,pre_72h_return,pre_72h_price,pre_72h_price_avg,pre_72h_volume,pre_72h_volume_avg,pre_72h_volume_sum,pre_72h_volume_tb,pre_72h_volume_tb_avg,pre_72h_volume_tb_sum,pre_72h_volume_quote,pre_72h_volume_quote_avg,pre_72h_volume_quote_sum,pre_72h_volume_quote_tb,pre_72h_volume_quote_tb_avg,pre_72h_volume_quote_tb_sum,pre_3d_market_cap_usd,pre_3d_market_cap_btc,pre_3d_price_usd,pre_3d_price_btc,pre_3d_volume_usd,pre_3d_volume_btc,pre_3d_twitter_index,pre_3d_reddit_index,pre_3d_alexa_index".split(",")

In [14]:
sample_df = pd.read_csv("test_sample.csv", names=column_list)

In [15]:
sample_df.head()

,,label,channel_id,coin,timestamp,length,coin_seq,feature_seq,pre_1h_return,pre_1h_price,pre_1h_price_avg,...,pre_72h_volume_quote_tb_sum,pre_3d_market_cap_usd,pre_3d_market_cap_btc,pre_3d_price_usd,pre_3d_price_btc,pre_3d_volume_usd,pre_3d_volume_btc,pre_3d_twitter_index,pre_3d_reddit_index,pre_3d_alexa_index
47611,47611,0,1136271991,LOOM,2019-06-04 12:01:33+00:00,0,NaN,NaN,0.025484,0.963094,0.963094,...,7.039568,25.700048,12.633567,7.872418e+03,0.917941,21.789446,8.723669,14.261000,15.273629,18.631966
34518,34518,0,1480242121,ARDR,2021-07-06 19:15:12+00:00,0,NaN,NaN,-0.003899,0.514174,0.514174,...,6.091853,27.296548,12.251924,1.649834e+04,0.487232,24.562623,9.515469,16.139953,12.767171,18.765943
57260,57260,0,1219293084,LOOM,2020-05-29 16:00:22+00:00,4,MDA\tBNT\tRLC\tGVT,4.2674.2584.66100000000000054.45954.500999...,0.006876,0.187372,0.187372,...,5.709533,23.835330,10.718100,1.797913e+03,0.202215,23.200714,10.082702,14.397280,15.273629,19.438315
10091,10091,0,1296134866,AKRO,2021-11-28 17:00:05+00:00,26,MTH\tEZ\tEVX\tWNXM\tEZ\tFXS\tVIB\tNAS\tEZ\tDRE...,0.0590.0590.1330.09150.12250.058989991597...,0.016018,0.056092,0.056092,...,4.793001,26.357949,10.554131,3.173270e+03,0.055483,22.821812,7.019359,15.580733,15.273629,17.772502
38637,38637,0,1267170242,KSM,2021-02-13 17:00:05+00:00,22,IDEX\tADX\tOAX\tARK\tRLC\tELF\tRLC\tBRD\tSTORJ...,0.1260.130.270.1650.19050.125823973771497...,0.008207,346.166343,346.166343,...,7.287367,30.130546,14.622910,1.308077e+07,280.736128,27.493693,11.985855,15.045337,15.273629,16.856475


In [3]:
input_file = "test_sample.csv" # pos_sample_fg.csv

feat_config = {
    "n_channel": 200,
    "d_channel": 16,
    "n_coin": 1000,
    "d_coin": 16,
    "n_feat": 142,
    "max_length": 50,
    "batch_size": 20,
    "epoch": 3
}

In [4]:
dataset = tf.data.TextLineDataset(input_file)

In [5]:
def parse_split(line):

    parse_res = tf.string_split([line], delimiter=",")
    values = parse_res.values
    label = values[0]
    channel = values[1]
    coin = values[2]
    length = values[4]
    coin_seq = values[5]
    feature_seq = values[6]
    feature_target = values[7:]

    return label, channel, coin, length, coin_seq, feature_seq, feature_target

dataset = dataset.map(parse_split, num_parallel_calls=2)

Instructions for updating:
delimiter is deprecated, please use sep instead.


In [6]:
dataset = dataset.shuffle(1).repeat(feat_config["epoch"]).batch(feat_config["batch_size"])

In [7]:
iterator = dataset.make_one_shot_iterator()

Instructions for updating:
This is a deprecated API that should only be used in TF 1 graph mode and legacy TF 2 graph mode available through `tf.compat.v1`. In all other situations -- namely, eager mode and inside `tf.function` -- you can consume dataset elements using `for elem in dataset: ...` or by explicitly creating iterator via `iterator = iter(dataset)` and fetching its elements via `values = next(iterator)`. Furthermore, this API is not available in TF 2. During the transition from TF 1 to TF 2 you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)` to create a TF 1 graph mode style iterator for a dataset created through TF 2 APIs. Note that this should be a transient state of your code base as there are in general no guarantees about the interoperability of TF 1 and TF 2 code.


In [8]:
label, channel, coin, length, coin_seq, feature_seq, feature_target = iterator.get_next()

<tf.Tensor 'IteratorGetNext:0' shape=(?,) dtype=string>

In [10]:
feature_target

<tf.Tensor 'IteratorGetNext:6' shape=(?, ?) dtype=string>

In [13]:
sess = tf.Session()
sess.run(channel)

InvalidArgumentError: Cannot batch tensors with different shapes in component 6. First element had shape [138] and element 2 had shape [140].
	 [[node IteratorGetNext (defined at <ipython-input-8-76da07df8976>:1) ]]

Errors may have originated from an input operation.
Input Source operations connected to node IteratorGetNext:
 OneShotIterator (defined at <ipython-input-7-5108a2f362bc>:1)

Original stack trace for 'IteratorGetNext':
  File "/Users/husihao/anaconda3/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/husihao/anaconda3/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/husihao/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/husihao/anaconda3/lib/python3.8/site-packages/traitlets/config/application.py", line 845, in launch_instance
    app.start()
  File "/Users/husihao/anaconda3/lib/pytho